In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# import fitbit download json file
df = pd.read_json (r'C:\Users\spice\Documents\Fitbit\AndrewJamieson\Physical Activity\heart_rate-2020-12-10.json')
df.head()

,dateTime,value
0,2020-12-10 08:00:01,"{'bpm': 65, 'confidence': 3}"
1,2020-12-10 08:00:06,"{'bpm': 63, 'confidence': 3}"
2,2020-12-10 08:00:16,"{'bpm': 64, 'confidence': 3}"
3,2020-12-10 08:00:21,"{'bpm': 68, 'confidence': 3}"
4,2020-12-10 08:00:26,"{'bpm': 73, 'confidence': 3}"
